In [95]:
from datasets import load_dataset, Dataset
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [3]:
ds = load_dataset("bigcode/jupyter-code-text-pairs", use_auth_token=True, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9305991 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/leandro/.cache/huggingface/datasets/bigcode___parquet/bigcode--jupyter-code-text-pairs-7dea51d66788f48c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [71]:
ds

Dataset({
    features: ['markdown', 'code', 'output', 'license', 'path', 'repo_name'],
    num_rows: 9305991
})

In [98]:
def build_content(markdown, code, output):
    if len(output)>1000:
        output_str = output[:1000] + "[...]"
    elif output == "_____no_output_____":
        output_str = "<empty_output>"
    else:
        output_str = output
    
    content = f"<jupyter_text>{markdown.strip()}<jupyter_code>{code.strip()}<jupyter_output>{output_str.strip()}"
    return content

current_repo_path = ds[0]["repo_name"] + "/" + ds[0]["path"]
current_content = "<jupyter_start>"
current_chain_length = 0

data_dict = {"content": [], "license": [ds[0]["license"]], "path": [ds[0]["path"]], "repo_name": [ds[0]["path"]], "chain_length": []}

for i in tqdm(range(int(len(ds)/1_000)+1)):
    end_index = min((i+1)*1000, len(ds))
    element = ds[i*1000:end_index]
    for j in range(min(1_000, end_index-i*1000)):
        repo_path = element["repo_name"][j] + "/" + element["path"][j]
    
        if repo_path != current_repo_path:
            data_dict["content"].append(current_content)
            data_dict["chain_length"].append(current_chain_length)
            data_dict["license"].append(element["license"][j])
            data_dict["path"].append(element["path"][j])
            data_dict["repo_name"].append(element["repo_name"][j])

            current_repo_path = repo_path
            current_content = "<jupyter_start>"
            current_chain_length = 0
        
        current_content += build_content(element["markdown"][j], element["code"][j], element["output"][j])
        current_chain_length +=1

data_dict["content"].append(current_content)
data_dict["chain_length"].append(current_chain_length)



100%|██████████| 9306/9306 [01:16<00:00, 122.06it/s]


In [99]:
len(data_dict["content"])

1045605

In [104]:
sum([len(content) for content in data_dict["content"]])/1000/1000/1000

11.187439308

In [100]:
print(np.mean(data_dict["chain_length"]))

8.900101854906968


In [102]:
ds_clean = Dataset.from_dict(data_dict)

In [103]:
ds_clean

Dataset({
    features: ['content', 'license', 'path', 'repo_name', 'chain_length'],
    num_rows: 1045605
})

In [105]:
ds_clean.push_to_hub("jupyter-structured-clean", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/23 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]